In [1]:
import pandas as pd
from pandas import read_csv
import numpy as np
import random as rnd
import re

# visualization
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')


# machine learning
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
#from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
#from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier



from collections import Counter
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
#from sklearn.metrics import ndcg_score
#from sklearn.metrics import label_ranking_average_precision_score
from matplotlib import pyplot

In [5]:
train = pd.read_csv('train_pre.csv', index_col = 0)
test = pd.read_csv('test_pre.csv', index_col = 0)
gender_df = pd.read_csv('gender_submission.csv', index_col = 0)
train.head()

,Survived,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,0,3,1,1,0,0,0,23,0,2,0,1
1,1,1,0,2,0,3,1,51,1,2,0,3
2,1,3,0,1,0,1,0,22,0,1,1,2
3,1,1,0,2,0,3,0,44,1,2,0,3
4,0,3,1,2,0,1,0,24,0,1,1,1


In [15]:
train['Survived'].describe()

count    891.000000
mean       0.383838
std        0.486592
min        0.000000
25%        0.000000
50%        0.000000
75%        1.000000
max        1.000000
Name: Survived, dtype: float64

In [18]:
X = train.drop('Survived', 1)
y = train['Survived']
X_train, X_test, y_train, y_test =  train_test_split(
    X, y, test_size=0.3, random_state=0)
X.head()

,Pclass,Sex,Age,Parch,Fare,Embarked,Name_length,Has_Cabin,FamilySize,IsAlone,Title
0,3,1,1,0,0,0,23,0,2,0,1
1,1,0,2,0,3,1,51,1,2,0,3
2,3,0,1,0,1,0,22,0,1,1,2
3,1,0,2,0,3,0,44,1,2,0,3
4,3,1,2,0,1,0,24,0,1,1,1


In [8]:
from sklearn.model_selection import cross_val_score
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1, random_state=42)
scores = cross_val_score(clf, X_train, y_train, cv=5
#                         , scoring='f1_macro'  # be default score is accuracy
                        )
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.81 accuracy with a standard deviation of 0.03


In [9]:
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing

clf = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))
scores = cross_val_score(clf, X, y, cv=5)
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores.mean(), scores.std()))

0.83 accuracy with a standard deviation of 0.02


In [12]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import accuracy_score

scores = cross_validate(clf, X, y,
                       scoring='accuracy', cv=5,
                       return_estimator=True, return_train_score=True)
sorted(scores.keys())
scores['train_score']

array([0.84831461, 0.8569425 , 0.84291725, 0.8513324 , 0.83730715])

In [13]:
print("%0.2f accuracy with a standard deviation of %0.2f" % (scores['test_score'].mean(), scores['test_score'].std()))

0.83 accuracy with a standard deviation of 0.02


In [20]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

# Set the parameters by cross-validation
tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
                     'C': [1, 10, 100, 1000]},
                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]
param_grid = [
  {'C': [1, 10, 100, 1000], 'kernel': ['linear']},
  {'C': [1, 10, 100, 1000], 'gamma': [0.001, 0.0001], 'kernel': ['rbf']},
 ]
scores = ['precision', 'recall']

for score in scores:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(
        svm.SVC(), param_grid, scoring='%s_macro' % score
    )
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision

Best parameters set found on development set:

{'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

Grid scores on development set:

0.811 (+/-0.049) for {'C': 1, 'kernel': 'linear'}
0.813 (+/-0.057) for {'C': 10, 'kernel': 'linear'}
0.811 (+/-0.049) for {'C': 100, 'kernel': 'linear'}
0.811 (+/-0.049) for {'C': 1000, 'kernel': 'linear'}
0.713 (+/-0.102) for {'C': 1, 'gamma': 0.001, 'kernel': 'rbf'}
0.714 (+/-0.113) for {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}
0.818 (+/-0.055) for {'C': 10, 'gamma': 0.001, 'kernel': 'rbf'}
0.732 (+/-0.083) for {'C': 10, 'gamma': 0.0001, 'kernel': 'rbf'}
0.821 (+/-0.060) for {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}
0.815 (+/-0.061) for {'C': 100, 'gamma': 0.0001, 'kernel': 'rbf'}
0.807 (+/-0.053) for {'C': 1000, 'gamma': 0.001, 'kernel': 'rbf'}
0.814 (+/-0.060) for {'C': 1000, 'gamma': 0.0001, 'kernel': 'rbf'}

Detailed classification report:

The model is trained on the full development set.
The scores are computed 

In [29]:
from pprint import pprint
from time import time
import logging
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

# pipeline defines estimator
pipe = Pipeline([
    ('rf',RandomForestClassifier(random_state = 42)),
    ('clf', SGDClassifier()),
])

parameters = {
    'rf__n_estimators':[120, 140],
    'rf__max_depth': [30, 50],
    'rf__min_samples_split': [2,3], 
    'rf__min_samples_leaf': [3, 5],  
    'rf__class_weight': [{0:1, 1:1}, {0:1, 1:5}, {0:1, 1:3}, 'balanced'], 
    'clf__max_iter': (20,),
    'clf__alpha': (0.00001, 0.000001),
    'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__max_iter': (10, 50, 80),
}

if __name__ == "__main__":
    # multiprocessing requires the fork to happen in a __main__ protected
    # block

    # find the best parameters for both the feature extraction and the
    # classifier
    grid_search = GridSearchCV(pipe, parameters, n_jobs=-1, verbose=1)

    print("Performing grid search...")
    print("pipeline:", [name for name, _ in pipe.steps])
    print("parameters:")
    pprint(parameters)
    t0 = time()
    grid_search.fit(X_train, y_train)
    print("done in %0.3fs" % (time() - t0))
    print()

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    y_true, y_pred = y_test, grid_search.predict(X_test)
    print(classification_report(y_true, y_pred))

TypeError: All intermediate steps should be transformers and implement fit and transform or be the string 'passthrough' 'RandomForestClassifier(random_state=42)' (type <class 'sklearn.ensemble._forest.RandomForestClassifier'>) doesn't